In [2]:
import numpy as np
import pandas as pd
import json
import requests
import urllib
import httplib
import base64

In [13]:
sample_text_array = ["Such a lovely hackathon we are in!", "We are definitely going to", "win and to go Paris!"]

sample_text = {}
for i in range(150):
    sample_text[i] = sample_text_array[i % 3]

# sample_text = {0 : "Such a lovely hackathon we are in!", 
#                1 : "We are definitely going to", 
#                2 : "win and to go Paris!"
#               }

pd_text = pd.DataFrame(pd.Series(sample_text).rename("text"))
pd_text.head()

,text
0,Such a lovely hackathon we are in!
1,We are definitely going to
2,win and to go Paris!
3,Such a lovely hackathon we are in!
4,We are definitely going to


In [4]:
api_endpoint = "https://westus.api.cognitive.microsoft.com/text/analytics/v2.0"
api_key_one = "28d6776ec31844a1aeb1095be8d99192"
api_key_two = "ec713e3e3ed64186aa185d02acbf5014"

In [5]:
def generate_json_request(pd_text):
    request = {}
    request["documents"] = []
    
    for index, row in pd_text.iterrows():
        phrase = {}
        phrase["lang"] = "en"
        phrase["id"] = index
        phrase["text"] = row["text"]
        request["documents"].append(phrase)
    return json.dumps(request)

# json_request = generate_json_request(pd_text)
# print json_request

In [6]:
def process_request(json_request):
    header = {
    'Accept' : 'application/json',
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': '28d6776ec31844a1aeb1095be8d99192',
    }
    try:
        conn = httplib.HTTPSConnection('westus.api.cognitive.microsoft.com')
        
        conn.request("POST", "/text/analytics/v2.0/sentiment", json_request, header)
        response = conn.getresponse()
        sentiment_data = response.read()
        
        conn.request("POST", "/text/analytics/v2.0/keyPhrases", json_request, header)
        response = conn.getresponse()
        key_phrase_data = response.read()
        
        conn.close()
        return sentiment_data, key_phrase_data
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
        return None    

In [7]:
# sentiment_data, key_phrase_data = process_request(json_request)

In [8]:
def add_text_analysis(base_df, sentiment_data, key_phrase_data):
    sentiment_dict = {}
    for entry in json.loads(sentiment_data)["documents"]:
        sentiment_dict[int(entry["id"])] = entry["score"]

    key_phrase_dict = {}
    for entry in json.loads(key_phrase_data)["documents"]:
        key_phrase_dict[int(entry["id"])] = entry["keyPhrases"]

    final_pd = pd.DataFrame(base_df)
    final_pd["sentiment_score"] = pd.Series(sentiment_dict)
    final_pd["key_phrases"] = pd.Series(key_phrase_dict)
    return final_pd

In [9]:
def main(base_df):
    json_request = generate_json_request(base_df)
    sentiment_data, key_phrase_data = process_request(json_request)
    final_df = add_text_analysis(base_df, sentiment_data, key_phrase_data)
    return final_pd

In [34]:
final_pd = main(pd_text)

In [35]:
final_pd

,text,sentiment_score,key_phrases
0,Such a lovely hackathon we are in!,0.967976,[lovely hackathon]
1,We are definitely going to,0.950928,[]
2,win and to go Paris!,0.870765,[win]


In [20]:
def topic_request(json_request):
    header = {
    'Accept' : 'application/json',
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': '28d6776ec31844a1aeb1095be8d99192',
    }
    try:
        conn = httplib.HTTPSConnection('westus.api.cognitive.microsoft.com')
        
        conn.request("POST", "/text/analytics/v2.0/topics", json_request, header)
        response = conn.getresponse()
        print response
        print response.getheader("operation-location")
        print response.status
        topic_data = response.read()
        print topic_data
        
        conn.close()
        return topic_data
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
        return None       

In [78]:
header = {'Accept' : 'application/json', 'Content-Type': 'application/json','Ocp-Apim-Subscription-Key': '28d6776ec31844a1aeb1095be8d99192'}
conn = httplib.HTTPSConnection('westus.api.cognitive.microsoft.com')
conn.request("POST", "/text/analytics/v2.0/topics", generate_json_request(pd_text), header)
response = conn.getresponse()
# print response.msg
print response.getheader("operation-location")
conn.close()

https://westus.api.cognitive.microsoft.com/text/analytics/v2.0/operations/fb1019d6ce8a4f2c999d0b423aa870f3


In [88]:
conn = httplib.HTTPSConnection('westus.api.cognitive.microsoft.com')
conn.request("GET", "/text/analytics/v2.0/operations/fb1019d6ce8a4f2c999d0b423aa870f3",
             "",
             header
            )
response = conn.getresponse()
print response.msg
print response.status
print response.read()
conn.close()

In [89]:
print response.msg
print response.status
print response.read()

Transfer-Encoding: chunked
Content-Type: application/json; charset=utf-8
x-ms-transaction-count: 0
x-aml-ta-request-id: fb5193a5-234c-4eef-bbb4-7c2643c84e2b
X-Content-Type-Options: nosniff
apim-request-id: eebae94e-2553-4005-bcc5-d2ab1dca35bf
Strict-Transport-Security: max-age=31536000; includeSubDomains; preload
Date: Sat, 24 Jun 2017 14:59:31 GMT

200
{"status":"Failed","createdDateTime":"2017-06-24T14:54:59Z","operationType":"topics","message":"Internal error while executing BES operation."}
